In [ ]:
import psycopg2, io, base64
from flask import Flask, render_template, request, url_for, redirect
import pandas as pd, matplotlib.pyplot as plt, plotly, json
import plotly.express as px
import matplotlib.ticker as mtick


app = Flask(__name__)

def bd(sql):
    conn = psycopg2.connect(host='localhost', database='IPA1', user='postgres', password='19122022')
    cur = conn.cursor()
    cur.execute(sql)
    dados = cur.fetchall()
    cur.close()
    conn.close()
    return dados



def gerar_plot(fig):
    img = io.BytesIO()
    plt.tight_layout()
    fig.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    plt.close(fig)
    return plot_url


@app.route('/', methods=('GET', 'POST'))
def autentic():
    if request.method == 'POST':
        conta = request.form['login']
        senha = request.form['senha']
        sql="select * from administrador.usuario where (login='"+conta+"' and senha='"+senha+"')"
        #sql="select * from administrador.usuario where (conta='"+conta+"')"
        record=bd(sql)
        nr=len(record)
        if nr>=1:
            return redirect(url_for('home'))
        else:
            return redirect(url_for('indexnulo'))
    return render_template('autentic.html')

@app.route('/nulo')
def indexnulo():
    return render_template('indexnulo.html')


@app.route('/home')
def home():
    sql = '''
    SELECT m.id_menu, m.name_menu, s.id_submenu, s.name_submenu
    FROM menu m
    LEFT JOIN submenu s ON s.id_menu = m.id_menu
    ORDER BY m.ordem, s.id_submenu;
    '''
    dados = bd(sql)

    submenu_links = {
        'Crimes': 'dashboard_crimes',
        'Faccionado': 'dashboard_faccionado',
        'Crimes Por Cidade': 'dashboard_cidades',
        'Por Cidade': 'dashboard_custos',
        'Criminosos': 'dashboard_criminosos',
        'Anual': 'dashboard_mes_ano',
        'Por Crime': 'dashboard_custos_por_crime',
        'Por Ano': 'dashboard_custos_temporais',
        'Geral': 'dashboard_geral',
        'Crime': 'relatorio_crime'
    }

    menu_dict = {}

    for id_menu, name_menu, id_submenu, name_submenu in dados:
        if name_menu not in menu_dict:
            menu_dict[name_menu] = []
        
        if name_submenu:
            rota = submenu_links.get(name_submenu, '#')
            menu_dict[name_menu].append((name_submenu, rota))
    
    return render_template('homepage.html', menus=menu_dict)


@app.route('/t1')
def index():
    sql='select * from menu'
    menus= bd(sql)
    return render_template('index.html', menus=menus)

@app.route('/t2')
def index1():
    sql='select * from submenu'
    menus= bd(sql)
    return render_template('index1.html', menus=menus)

@app.route('/mapa')
def index3():
    sql = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia) AS total_ocorrencias
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime
        ORDER BY total_ocorrencias DESC;
    '''
    dados = bd(sql)

    nomes_crimes = [linha[0] for linha in dados]
    totais = [linha[1] for linha in dados]

    df = pd.DataFrame({
        'Crime': nomes_crimes,
        'Total': totais
    })


    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Crime'], df['Total'], color='skyblue')
    ax.set_title('Quantidade de Ocorrências por Crime')
    ax.set_xlabel('Tipo de Crime')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    img = io.BytesIO()
    plt.tight_layout()
    plt.savefig(img, format='png')
    img.seek(0)
    plt.close(fig)

    plot_url = base64.b64encode(img.getvalue()).decode()

    return render_template('index3.html', plot_url=plot_url)



@app.route('/dashboard/crimes')
def dashboard_crimes():
    sql = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia) AS total
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime
        ORDER BY total DESC;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Crime', 'Total'])

    fig,ax = plt.subplots(figsize=(10,6))
    ax.bar(df['Crime'],df['Total'],color='skyblue')
    ax.set_title('Ocorrências por Tipo de Crime')
    ax.set_xlabel('Tipo de Crime')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)

    return render_template("grafico_crimes.html", plot_url=plot_url, titulo="Crimes por Tipo")


@app.route('/dashboard/faccionado')
def dashboard_faccionado():
    sql = '''
        SELECT faccionado::text, COUNT(*) 
        FROM criminoso
        GROUP BY faccionado;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Faccionado', 'Total'])

    labels = ['Faccionado' if f == 'true' else 'Não Faccionado' for f in df['Faccionado']]
    fig, ax = plt.subplots()
    ax.pie(df['Total'], autopct='%1.1f%%', startangle=140)
    ax.set_title('Distribuição de Faccionados')
    ax.legend(labels,title='Categoria')

    plot_url = gerar_plot(fig)
    return render_template("grafico_faccionado.html", plot_url=plot_url,titulo='Criminosos Faccionados')


@app.route('/dashboard/cidades')
def dashboard_cidades():
    sql = '''
        SELECT cidade, COUNT(*) 
        FROM ocorrencia 
        GROUP BY cidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Cidade', 'Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Cidade'], df['Total'], color='orange')
    ax.set_title('Crimes por Cidade')
    ax.set_xlabel('Cidade')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_cidades.html", plot_url=plot_url, titulo='Crimes por Cidades')


@app.route('/dashboard/custos')
def dashboard_custos():
    sql = '''
        SELECT cidade, SUM(custo_operacional)
        FROM ocorrencia
        GROUP BY cidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Cidade', 'Custo Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Cidade'], df['Custo Total'], color='green')
    ax.set_title('Custo Operacional por Cidade')
    ax.set_xlabel('Cidade')
    ax.set_ylabel('Custo Total')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_custos.html", plot_url=plot_url, titulo='Custo gasto por Cidade')


@app.route('/dashboard/naturalidade')
def dashboard_criminosos():
    sql = '''
        SELECT naturalidade, COUNT(*)
        FROM criminoso
        GROUP BY naturalidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Naturalidade', 'Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Naturalidade'], df['Total'], color='purple')
    ax.set_title('Criminosos por Naturalidade')
    ax.set_xlabel('Naturalidade')
    ax.set_ylabel('Total')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_criminosos.html", plot_url=plot_url, titulo='Naturalidade Criminosos')

@app.route('/dashboard/mes_ano')
def dashboard_mes_ano():
    sql = '''
        SELECT 
            TO_CHAR(o.data_ocorrencia, 'YYYY-MM') AS ano_mes,
            COUNT(*) AS total
        FROM crime_ocorrencia co
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        WHERE o.data_ocorrencia IS NOT NULL
        GROUP BY ano_mes
        ORDER BY ano_mes;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Ano_Mês', 'Total'])

    df['Mês_Ano'] = pd.to_datetime(df['Ano_Mês']).dt.strftime('%b/%Y')

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df['Mês_Ano'], df['Total'], color='purple')
    ax.fill_between(df['Mês_Ano'], df['Total'], color='plum', alpha=0.3)


    ax.set_title('Crimes por Mês e Ano')
    ax.set_xlabel('Mês/Ano')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_mes_ano.html", plot_url=plot_url, titulo='Crimes por Mês e Ano')


@app.route('/dashboard/custos_por_crime')
def dashboard_custos_por_crime():
    sql = '''
        SELECT c.nome_crime, SUM(o.custo_operacional) AS custo_total
        FROM crime_ocorrencia co JOIN crime c ON co.id_crime = c.id_crime
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        GROUP BY c.nome_crime;

    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Crime', 'Custo Total'])

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.barh(df['Crime'], df['Custo Total'], color='red')
    ax.set_title('Custo Operacional por Tipo de Crime')
    ax.set_xlabel('Custo Total')
    ax.set_ylabel('Tipo de Crime')
    ax.invert_yaxis()

    plt.tight_layout()
    plot_url = gerar_plot(fig)
    return render_template("grafico_custos_por_crime.html", plot_url=plot_url, titulo='Custos por Crime')


@app.route('/dashboard/custos_temporais')
def dashboard_custos_temporais():
    sql = '''
        SELECT 
    EXTRACT(YEAR FROM o.data_ocorrencia)::int AS ano,
    TO_CHAR(o.data_ocorrencia, 'MM') AS mes,
    SUM(o.custo_operacional) AS total
    FROM 
    ocorrencia o
    WHERE 
    o.data_ocorrencia IS NOT NULL AND 
    EXTRACT(YEAR FROM o.data_ocorrencia) IN (2021, 2022)
    GROUP BY 
    ano, mes
    ORDER BY 
    ano, mes;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Ano', 'Mes', 'Total'])

    df_pivot = df.pivot(index='Mes', columns='Ano', values='Total').fillna(0)
    df_pivot = df_pivot.sort_index()

    meses = df_pivot.index
    largura = 0.35
    x = range(len(meses))

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar([i - largura/2 for i in x], df_pivot[2021], width=largura, label='2021', color='royalblue')
    ax.bar([i + largura/2 for i in x], df_pivot[2022], width=largura, label='2022', color='seagreen')

    ax.set_title('Comparativo de Custos Operacionais: 2021 vs 2022')
    ax.set_xlabel('Mês')
    ax.set_ylabel('Custo Total')
    ax.set_xticks(list(x))
    ax.set_xticklabels(meses)
    ax.legend()

    plt.tight_layout()
    plot_url = gerar_plot(fig)

    return render_template('dashboard_custos_temporais.html', plot_url=plot_url, titulo='Comparativo 2021 vs 2022')


@app.route('/bi')
def dashboard_geral():
    # Lista de (titulo, SQL, tipo)
    graficos = []

    # Crimes por tipo
    sql1 = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia)
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime
        ORDER BY 2 DESC;
    '''
    df1 = pd.DataFrame(bd(sql1), columns=['Crime', 'Total'])
    fig1, ax1 = plt.subplots(figsize=(6, 4))
    ax1.bar(df1['Crime'], df1['Total'], color='royalblue')
    ax1.set_title('Ocorrências por Tipo de Crime')
    ax1.tick_params(axis='x', rotation=45)
    graficos.append(('Ocorrências por Tipo de Crime', gerar_plot(fig1)))

    # Faccionados
    sql2 = '''
        SELECT faccionado::text, COUNT(*) 
        FROM criminoso
        GROUP BY faccionado;
    '''
    df2 = pd.DataFrame(bd(sql2), columns=['Faccionado', 'Total'])
    fig2, ax2 = plt.subplots()
    ax2.pie(df2['Total'], labels=df2['Faccionado'], autopct='%1.1f%%', startangle=90)
    ax2.set_title('Distribuição de Faccionados')
    graficos.append(('Distribuição de Faccionados', gerar_plot(fig2)))

    # Crimes por cidade
    sql3 = '''
        SELECT cidade, COUNT(*) 
        FROM ocorrencia 
        GROUP BY cidade
        ORDER BY 2 DESC;
    '''
    df3 = pd.DataFrame(bd(sql3), columns=['Cidade', 'Total'])
    fig3, ax3 = plt.subplots(figsize=(6, 4))
    ax3.bar(df3['Cidade'], df3['Total'], color='darkorange')
    ax3.set_title('Crimes por Cidade')
    ax3.tick_params(axis='x', rotation=45)
    graficos.append(('Crimes por Cidade', gerar_plot(fig3)))

    # Custo por cidade
    sql4 = '''
        SELECT cidade, SUM(custo_operacional)
        FROM ocorrencia
        GROUP BY cidade
        ORDER BY 2 DESC;
    '''
    df4 = pd.DataFrame(bd(sql4), columns=['Cidade', 'Custo Total'])
    fig4, ax4 = plt.subplots(figsize=(6, 4))
    ax4.bar(df4['Cidade'], df4['Custo Total'], color='seagreen')
    ax4.set_title('Custo Operacional por Cidade')
    ax4.tick_params(axis='x', rotation=45)
    graficos.append(('Custo Operacional por Cidade', gerar_plot(fig4)))

    # Crimes por mês e ano
    sql5 = '''
        SELECT TO_CHAR(o.data_ocorrencia, 'YYYY-MM') AS ano_mes, COUNT(*)
        FROM crime_ocorrencia co
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        WHERE o.data_ocorrencia IS NOT NULL
        GROUP BY ano_mes ORDER BY ano_mes;
    '''
    df5 = pd.DataFrame(bd(sql5), columns=['Ano_Mês', 'Total'])
    df5['Mês_Ano'] = pd.to_datetime(df5['Ano_Mês']).dt.strftime('%b/%Y')
    fig5, ax5 = plt.subplots(figsize=(8, 4))
    ax5.plot(df5['Mês_Ano'], df5['Total'], marker='o', color='purple')
    ax5.set_title('Crimes por Mês e Ano')
    ax5.tick_params(axis='x', rotation=45)
    graficos.append(('Crimes por Mês e Ano', gerar_plot(fig5)))


    return render_template("index2.html", graficos=graficos)


@app.route('/relatorio/crime')
def relatorio_crime():
    sql = '''
        SELECT 
            c.nome_crime, 
            COUNT(co.id_ocorrencia) AS total_ocorrencias,
            SUM(o.tempo_prisao) AS tempo_total_prisao,
            AVG(o.tempo_prisao) AS media_prisao,
            SUM(o.custo_operacional) AS custo_total
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        GROUP BY c.nome_crime;
    '''
    dados = bd(sql)
    colunas = ['Crime', 'Ocorrências', 'Tempo Total de Prisão', 'Tempo Médio de Prisão', 'Custo Total']
    return render_template("relatorio_crime.html", dados=dados, colunas=colunas, titulo="Relatório por Tipo de Crime")





if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2025-07-09 09:52:39,012] ERROR in app: Exception on /relatorio/crime [GET]
Traceback (most recent call last):
  File "c:\Users\freit\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\freit\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\freit\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\freit\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\freit\A